# How to save and restore a trained model

After scrolling through the posts of [reddit.com/r/learnmachinelearning](https://www.reddit.com/r/learnmachinelearning/), I've realized that the major bottlenecks of a machine learning project occur in the data input pipeline and in the final stage of the model, where you have to save the model and make predictions on new data.
So I thought that it would be useful to make a simple and straightforward tutorial to show you how you could save and restore a model that you have built with Tensorflow Eager.

### Tutorial flowchart
----

![img](tutorials_graphics/save_restore_model.png)

## Import here useful libraries

In [2]:
# Import TensorFlow and TensorFlow Eager
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# Import function to generate toy classication problem
from sklearn.datasets import make_moons

In [3]:
# Enable eager mode. Once activated it cannot be reversed! Run just once.
tfe.enable_eager_execution()

## Part I: Build a simple neural network model for binary classification

In [4]:
class simple_nn(tf.keras.Model):
    def __init__(self):
        super(simple_nn, self).__init__()
        """ Define here the layers used during the forward-pass 
            of the neural network.
        """   
        # Hidden layer.
        self.dense_layer = tf.layers.Dense(10, activation=tf.nn.relu)
        # Output layer. No activation.
        self.output_layer = tf.layers.Dense(2, activation=None)
    
    def predict(self, input_data):
        """ Runs a forward-pass through the network.     
            Args:
                input_data: 2D tensor of shape (n_samples, n_features).   
            Returns:
                logits: unnormalized predictions.
        """
        hidden_activations = self.dense_layer(input_data)
        logits = self.output_layer(hidden_activations)
        return logits
    
    def loss_fn(self, input_data, target):
        """ Defines the loss function used during 
            training.         
        """
        logits = self.predict(input_data)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=target, logits=logits)
        return loss
    
    def grads_fn(self, input_data, target):
        """ Dynamically computes the gradients of the loss value
            with respect to the parameters of the model, in each
            forward pass.
        """
        with tfe.GradientTape() as tape:
            loss = self.loss_fn(input_data, target)
        return tape.gradient(loss, self.variables)
    
    def fit(self, input_data, target, optimizer, num_epochs=500, verbose=50):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs.
        """
        for i in range(num_epochs):
            grads = self.grads_fn(input_data, target)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.loss_fn(input_data, target).numpy()))

## Part II: Train model 

In [9]:
# Generate toy dataset for classification
# X is a matrix of n_samples x n_features and represents the input features
# y is a vector with length n_samples and represents our targets
X, y = make_moons(n_samples=100, noise=0.1, random_state=2018)
X_train, y_train = tf.constant(X[:80,:]), tf.constant(y[:80])
X_test, y_test = tf.constant(X[80:,:]), tf.constant(y[80:])

In [10]:
optimizer = tf.train.GradientDescentOptimizer(5e-1)
model = simple_nn()
model.fit(X_train, y_train, optimizer, num_epochs=500, verbose=50)

Loss at epoch 1: 0.658276
Loss at epoch 50: 0.302146
Loss at epoch 100: 0.268594
Loss at epoch 150: 0.247425
Loss at epoch 200: 0.229143
Loss at epoch 250: 0.197839
Loss at epoch 300: 0.143365
Loss at epoch 350: 0.098039
Loss at epoch 400: 0.070781
Loss at epoch 450: 0.053753
Loss at epoch 500: 0.042401


## Part III: Save trained model

In [11]:
# Specify checkpoint directory
checkpoint_directory = 'models_checkpoints/SimpleNN/'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [12]:
# Save trained model
checkpoint.save(file_prefix=checkpoint_directory)

'models_checkpoints/SimpleNN/-1'

## Part IV: Restore trained model


In [13]:
# Reinitialize model instance
model = simple_nn()
optimizer = tf.train.GradientDescentOptimizer(5e-1)

In [14]:
# Specify checkpoint directory
checkpoint_directory = 'models_checkpoints/SimpleNN/'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [15]:
# Restore model from latest chekpoint
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

## Part V: Check if the model was restored correctly

In [16]:
model.fit(X_train, y_train, optimizer, num_epochs=1)

Loss at epoch 1: 0.042220


The loss seems to be consistent with the loss we obtained in the last epoch of previous training :)!

## Part VI: Make predictions on new data

In [17]:
logits_test = model.predict(X_test)

In [18]:
print(logits_test)

tf.Tensor(
[[ 1.54352813 -0.83117302]
 [-1.60523365  2.82397487]
 [ 2.87589525 -1.36463485]
 [-1.39461001  2.62404279]
 [ 0.82305161 -0.55651397]
 [ 3.53674391 -2.55593046]
 [-2.97344627  3.46589599]
 [-1.69372442  2.95660466]
 [-1.43226137  2.65357974]
 [ 3.11479995 -1.31765645]
 [-0.65841567  1.60468631]
 [-2.27454367  3.60553595]
 [-1.50170912  2.74410115]
 [ 0.76261479 -0.44574208]
 [ 2.34516959 -1.6859307 ]
 [ 1.92181942 -1.63766352]
 [ 4.06047684 -3.03988941]
 [ 1.00252324 -0.78900484]
 [ 2.79802993 -2.2139734 ]
 [-1.43933035  2.68037059]], shape=(20, 2), dtype=float64)
